In [ ]:
from notebook import Notebook
import os
import shutil
from string import Template
import sys
from io import StringIO


In [ ]:
if len(sys.argv)<2:
    print('TODO gracefully shutdown')
    
notebook_path = sys.argv[1]
#notebook_path = os.environ.get('notebook_path','../component-library/input/input-url.ipynb')

nb = Notebook(notebook_path)

In [ ]:
name = nb.get_name()
description = nb.get_description()
inputs = nb.get_inputs()
outputs = nb.get_outputs()
requirements = nb.get_requirements()

In [ ]:
print(name)
print(description)
print(inputs)
print(outputs)
print(requirements)

In [ ]:
!echo {notebook_path}

In [ ]:
!jupyter nbconvert --to script `echo {notebook_path}` 

In [ ]:
target_code = notebook_path.replace('.ipynb','.py').split('/')[-1:][0]
shutil.move(notebook_path.replace('.ipynb','.py'),target_code)

In [ ]:
requirements_docker = list(map(lambda s: 'RUN '+s, requirements))

In [ ]:
docker_file = """FROM registry.access.redhat.com/ubi8/python-39
{}
ADD {} .
ENTRYPOINT ["python", "{}"]
""".format(
    '\n'.join(requirements_docker),
    target_code,
    target_code
)
with open("Dockerfile", "w") as text_file:
    text_file.write(docker_file)
!cat Dockerfile


In [ ]:
!docker build -t `echo claimed-{name}` .
!docker tag  `echo claimed-{name}` `echo romeokienzler/claimed-{name}`
!docker push `echo romeokienzler/claimed-{name}`

In [ ]:
def get_inputs():
    with StringIO() as inputs_str:
        for input_key, input_value in inputs.items():
            t = Template("- {name: $name, type: $type, description: '$description'}")
            print(t.substitute(name=input_key, type=input_value[1], description=input_value[0]), file=inputs_str)
        return inputs_str.getvalue()

In [ ]:
def get_outputs():
    with StringIO() as outputs_str:
        assert len(outputs) == 1, 'exactly one output currently supported: '+ str((len(outputs.items())))
        for output_key, output_value in outputs.items():
            t = Template("- {name: $name, type: $type, description: '$description'}")
            print(t.substitute(name=output_key, type=output_value[1], description=output_value[0]), file=outputs_str)
        return outputs_str.getvalue()


In [ ]:
def get_output_name():
    for output_key, output_value in outputs.items():
        return output_key

In [ ]:

def get_input_for_implementation():
    with StringIO() as inputs_str:
        for input_key, input_value in inputs.items():
            t = Template("        - {inputValue: $name}")
            print(t.substitute(name=input_key), file=inputs_str)
        return inputs_str.getvalue()    

In [ ]:
def get_parameter_list():
    return_value = str()
    index = 0
    for output_key, output_value in outputs.items():
        return_value = return_value + output_key + '="$' + str(index) + '" '
        index = index + 1
    for input_key, input_value in inputs.items():
        return_value = return_value + input_key + '="$' + str(index) + '" '
        index = index + 1
    return return_value      

In [ ]:

t = Template('''name: $name
description: $description

inputs:
$inputs

outputs:
$outputs

implementation:
    container:
        image: $container_uri
        command:
        - sh
        - -ec
        - |
          python $call
        - {outputPath: $outputPath}
$input_for_implementation''')
yaml = t.substitute(
            name=name,
            description=description,
            inputs=get_inputs(),
            outputs=get_outputs(),
            container_uri=f"romeokienzler/claimed-{name}",
            outputPath=get_output_name(),
            input_for_implementation=get_input_for_implementation(),
            call=f'./{target_code} {get_parameter_list()}' 
        )
print(yaml)

In [ ]:
target_yaml_path = notebook_path.replace('.ipynb','.yaml')

with open(target_yaml_path, "w") as text_file:
    text_file.write(yaml)